### Pre Processor

In [4]:
## Import all necessary libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import pickle
import warnings
warnings.filterwarnings("ignore")


In [5]:
## load the dataset
df = pd.read_csv('qa_test_cases.csv')

In [6]:
df.head()

,Test_Case_ID,Functionality,Story_Type,Core_Function,Affects_Workflow,Has_Workaround,Impact_Level,Severity_Score,Tester_Name,Time_Taken_Minutes,Priority_Label
0,TC001,Login,React Story,No,No,Yes,NaN,2,Charlie,22,No Issues
1,TC002,Payment Gateway,AEM Story,Yes,No,Yes,Low,2,Alice,30,P3
2,TC003,Search,React Story,Yes,Yes,Yes,NaN,0,Charlie,19,No Issues
3,TC004,Checkout,React Story,No,Yes,Yes,NaN,5,Charlie,22,No Issues
4,TC005,Profile Management,React Story,Yes,No,Yes,NaN,0,Alice,28,No Issues


In [7]:
## Drop unnecessary columns
df.drop(columns=['Test_Case_ID','Impact_Level'], inplace=True)

In [8]:
df.head()

,Functionality,Story_Type,Core_Function,Affects_Workflow,Has_Workaround,Severity_Score,Tester_Name,Time_Taken_Minutes,Priority_Label
0,Login,React Story,No,No,Yes,2,Charlie,22,No Issues
1,Payment Gateway,AEM Story,Yes,No,Yes,2,Alice,30,P3
2,Search,React Story,Yes,Yes,Yes,0,Charlie,19,No Issues
3,Checkout,React Story,No,Yes,Yes,5,Charlie,22,No Issues
4,Profile Management,React Story,Yes,No,Yes,0,Alice,28,No Issues


In [9]:
## Level-encode categorical variables
label_encoder=LabelEncoder()
categorical_cols = ['Priority_Label', 'Core_Function', 'Affects_Workflow', 'Has_Workaround']
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

In [10]:
df.head()

,Functionality,Story_Type,Core_Function,Affects_Workflow,Has_Workaround,Severity_Score,Tester_Name,Time_Taken_Minutes,Priority_Label
0,Login,React Story,0,0,1,2,Charlie,22,0
1,Payment Gateway,AEM Story,1,0,1,2,Alice,30,3
2,Search,React Story,1,1,1,0,Charlie,19,0
3,Checkout,React Story,0,1,1,5,Charlie,22,0
4,Profile Management,React Story,1,0,1,0,Alice,28,0


In [11]:
## One-hot encode the 'Functionality' column

onehot_encoder_func=OneHotEncoder()
func_encoder=onehot_encoder_func.fit_transform(df[['Functionality']])

In [12]:
onehot_encoder_func.get_feature_names_out(['Functionality'])

array(['Functionality_Checkout', 'Functionality_Login',
       'Functionality_Payment Gateway',
       'Functionality_Profile Management', 'Functionality_Search'],
      dtype=object)

In [13]:
func_encoded_df=pd.DataFrame(func_encoder.toarray(),columns=onehot_encoder_func.get_feature_names_out(['Functionality']))
func_encoded_df.head()

,Functionality_Checkout,Functionality_Login,Functionality_Payment Gateway,Functionality_Profile Management,Functionality_Search
0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0


In [14]:
## Combine the func encoded columns with the original dataset
df=pd.concat([df.drop('Functionality',axis=1),func_encoded_df],axis=1) 
df.head()

,Story_Type,Core_Function,Affects_Workflow,Has_Workaround,Severity_Score,Tester_Name,Time_Taken_Minutes,Priority_Label,Functionality_Checkout,Functionality_Login,Functionality_Payment Gateway,Functionality_Profile Management,Functionality_Search
0,React Story,0,0,1,2,Charlie,22,0,0.0,1.0,0.0,0.0,0.0
1,AEM Story,1,0,1,2,Alice,30,3,0.0,0.0,1.0,0.0,0.0
2,React Story,1,1,1,0,Charlie,19,0,0.0,0.0,0.0,0.0,1.0
3,React Story,0,1,1,5,Charlie,22,0,1.0,0.0,0.0,0.0,0.0
4,React Story,1,0,1,0,Alice,28,0,0.0,0.0,0.0,1.0,0.0


In [15]:
## One-hot encode the 'Story_Type' column

onehot_encoder_ST=OneHotEncoder()
ST_encoder=onehot_encoder_ST.fit_transform(df[['Story_Type']])

onehot_encoder_ST.get_feature_names_out(['Story_Type'])

ST_encoded_df=pd.DataFrame(ST_encoder.toarray(),columns=onehot_encoder_ST.get_feature_names_out(['Story_Type']))

df=pd.concat([df.drop('Story_Type',axis=1),ST_encoded_df],axis=1) 

df.head()

,Core_Function,Affects_Workflow,Has_Workaround,Severity_Score,Tester_Name,Time_Taken_Minutes,Priority_Label,Functionality_Checkout,Functionality_Login,Functionality_Payment Gateway,Functionality_Profile Management,Functionality_Search,Story_Type_AEM Story,Story_Type_API Gateway Changes,Story_Type_Database Update,Story_Type_React Story,Story_Type_UI Enhancement
0,0,0,1,2,Charlie,22,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,1,2,Alice,30,3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1,1,0,Charlie,19,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0,1,1,5,Charlie,22,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0,1,0,Alice,28,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
## One-hot encode the 'Tester_Name' column

onehot_encoder_TN=OneHotEncoder()
TN_encoder=onehot_encoder_TN.fit_transform(df[['Tester_Name']])

onehot_encoder_TN.get_feature_names_out(['Tester_Name'])

TN_encoded_df=pd.DataFrame(TN_encoder.toarray(),columns=onehot_encoder_TN.get_feature_names_out(['Tester_Name']))

df=pd.concat([df.drop('Tester_Name',axis=1),TN_encoded_df],axis=1) 

df.head()

,Core_Function,Affects_Workflow,Has_Workaround,Severity_Score,Time_Taken_Minutes,Priority_Label,Functionality_Checkout,Functionality_Login,Functionality_Payment Gateway,Functionality_Profile Management,Functionality_Search,Story_Type_AEM Story,Story_Type_API Gateway Changes,Story_Type_Database Update,Story_Type_React Story,Story_Type_UI Enhancement,Tester_Name_Alice,Tester_Name_Bob,Tester_Name_Charlie
0,0,0,1,2,22,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1,0,1,2,30,3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,1,1,0,19,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0,1,1,5,22,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1,0,1,0,28,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [17]:
## Save all encoders 
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

with open('onehot_encoder_func.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_func, f)

with open('onehot_encoder_ST.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_ST, f)

with open('onehot_encoder_TN.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_TN, f)

In [18]:
## Divide the dataset into features and target variable

X = df.drop('Priority_Label', axis=1)
y = df['Priority_Label']

## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

## Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
## Save the scaler

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [20]:
y_test.unique()

array([3, 0, 2, 1])

### ANN Implementation

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [27]:
## Build the ANN model

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## Input layer with 64 neurons
    Dense(32, activation='relu'), ## Hidden layer with 32 neurons
    Dense(4, activation='softmax') ## Output layer for binary classification
])

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                1216      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 4)                 132       
                                                                 
Total params: 3428 (13.39 KB)
Trainable params: 3428 (13.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001) ## Define the optimizer

In [36]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',   ## multi-class one-hot labels
    metrics=['accuracy']
)

In [37]:
## Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [38]:
## Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,
                    callbacks=[early_stopping])

Epoch 1/100


5/5 [==============================] - 1s 59ms/step - loss: 1.2112 - accuracy: 0.4750 - val_loss: 1.1002 - val_accuracy: 0.6500
Epoch 2/100
5/5 [==============================] - 0s 10ms/step - loss: 0.9785 - accuracy: 0.7500 - val_loss: 0.9586 - val_accuracy: 0.6750
Epoch 3/100
5/5 [==============================] - 0s 7ms/step - loss: 0.8077 - accuracy: 0.8188 - val_loss: 0.8683 - val_accuracy: 0.7250
Epoch 4/100
5/5 [==============================] - 0s 8ms/step - loss: 0.6728 - accuracy: 0.8562 - val_loss: 0.8253 - val_accuracy: 0.7250
Epoch 5/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5927 - accuracy: 0.8562 - val_loss: 0.8066 - val_accuracy: 0.7250
Epoch 6/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5414 - accuracy: 0.8562 - val_loss: 0.8025 - val_accuracy: 0.7250
Epoch 7/100
5/5 [==============================] - 0s 7ms/step - loss: 0.5045 - accuracy: 0.8562 - val_loss: 0.8032 - val_accuracy: 0.7250
Epoch 8/100
5/5 [======

In [39]:
model.save('model.h5')  ## Save the trained model